# Применение линейных методов классификации для диагностики кризиса теплообмена в ЯЭУ.
Группа: ИВТ-М20.<br/>
Студент: Лискунов Роман Геннадьвич

In [28]:
from typing import Union, Any

from pandas import (
    DataFrame,
    read_csv
)
from sklearn.metrics import (
    accuracy_score,
    classification_report
)
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV
)
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


## 1. Прочтите данные из файлов

In [29]:
transformators: DataFrame = DataFrame(
    read_csv(
        'data/transformators.csv',
        sep=',',
        header=0
    )
)

In [30]:
x = transformators.drop('label', axis=1)
y = transformators.label

## 2. Отобразите несколько первых и несколько последних записей.

In [38]:
print(transformators.head())
print('\n')
print(transformators.tail())

         H2        CO      C2H4      C2H2  label
0  0.002590  0.016491  0.002876  0.000336      1
1  0.002825  0.014728  0.001992  0.000339      1
2  0.002764  0.017030  0.003394  0.000195      1
3  0.000508  0.016334  0.004104  0.000380      1
4  0.002654  0.021278  0.004068  0.000124      1


            H2        CO      C2H4      C2H2  label
2095  0.002549  0.009460  0.007465  0.000242      1
2096  0.000915  0.005770  0.007230  0.000081      3
2097  0.001565  0.002492  0.008529  0.000176      3
2098  0.001688  0.020291  0.007305  0.000252      1
2099  0.002935  0.028001  0.008048  0.000219      1


## 3. Разбейте данные для классификации

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, train_size=0.7)

##  4. Примените метод опорных векторов для классификации (SVC)

In [33]:
clf: Pipeline = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(x, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

## 5. Сделайте предсказание на тестовой выборке

In [37]:
pred = clf.predict(x_test)
print(f"prediction: {pred}", end='\n\n')
print(f"accuracy_score: {round(accuracy_score(y_test, pred, normalize=True), 3)}", end='\n\n')
print(f"classification_report: {classification_report(y_test, pred)}", end='\n\n')

prediction: [1 1 1 4 4 1 1 3 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1
 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 4 1 1 3 1 1 1 2 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1
 1 1 1 1 2 1 1 3 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1
 4 2 1 1 1 3 1 2 1 4 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 3 1 1 1 1 1 1 4 1 1 1
 4 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1
 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 4 1
 4 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 4 1 1 4 3 1 2 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 4 1 1 1 1 4 1 1 1 1 1 1 2 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 4 4 1 1 1
 1 1 1 1 1 1 4 1 1 4 1 1 1 1 1 1 4 1 1 2 1 1 1 1 3 1 4 1 1 1 1 1 1 1 1 4 1
 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 4 4 1 1 3 1 1 2 1 1 1 1 1 1 1 1
 1 1 3 1 3 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 4 1 4 1 1 1 1 1 1 1 1
 4 1 1 4 4 1 

## 6. Подбор гиперпараметров

In [35]:
parameters: dict[str, Union[tuple[str, str], list[int]]] = {
    'kernel': ('linear', 'rbf'),
    'C': [1, 10]
}

grid: GridSearchCV = GridSearchCV(
    SVC(),
    parameters,
    cv=5
)
results: Union[GridSearchCV, Any] = grid.fit(x_train, y_train)
print(results)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})


## 7. Заново обучим модель с подобранными гиперпараметрами

In [36]:
print(f"params: {results.best_params_}", end='\n\n')
print(f"score: {round(results.best_score_, 3)}", end='\n\n')
print(f"prediction: {results.predict(x_test)}", end='\n\n')

pred = results.predict(x_test)
print(f"accuracy_score: {round(accuracy_score(y_test, pred, normalize=True), 3)}")

params: {'C': 10, 'kernel': 'rbf'}

score: 0.927

prediction: [1 1 1 4 4 1 1 3 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1
 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 4 1 1 3 1 1 1 4 1 1 4 1 1 3 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1
 1 1 1 1 2 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1
 2 2 1 1 1 3 1 2 1 4 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 3
 4 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1
 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 4 3
 4 1 1 1 1 1 1 1 1 1 2 1 4 1 1 1 1 1 1 4 1 1 4 3 1 4 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 4 1 1 1 1 4 1 3 1 1 1 1 2 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 4 1 1 1 1
 1 1 1 1 1 1 4 1 1 4 1 1 1 1 1 1 4 1 1 4 1 1 1 1 3 1 2 1 1 1 1 1 1 1 1 2 1
 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 4 1 1 1 1 2 4 1 1 3 1 1 2 1 1 1 1 1 1 1 1
 1 1 3 1 3 1 1 1 1 1 1 1 1 1 1 1 3 1 1

# Ответы на контрольные вопросы

## 1. Опишите этапы построения алгоритма классификации SVC. В чем его отличие от других линейных алгоритмов?

## 2. Какие гиперпараметры влияют на качество работы классификационной модели опорных векторов SVC? Каким образом их можно настроить?

## 3. Опишите параметры лучшей модели, полученной в лабораторной работе.

